In [10]:
import sympy as sp, numpy as np, gurobipy as gb, pandas as pd

In [11]:
import sympy as sp
from gurobipy import Model, LinExpr, QuadExpr, GRB


# Define a function to convert SymPy expression to Gurobi expression
def sympy_to_gurobi(sympy_expr, symbol_map, model, aux_var_count=[0]):
    """
    Recursively convert a SymPy expression to a Gurobi expression, 
    
    handling exponentials, powers, divisions, and other complex expressions with auxiliary variables and constraints.
    
    Parameters:
        sympy_expr (sp.Expr): SymPy expression to convert.
        symbol_map (dict): Mapping from SymPy symbols to Gurobi variables.
        model (gurobipy.Model): Gurobi model to add constraints for complex expressions.
        aux_var_count (list): A list to keep track of the auxiliary variable count.
        
    Returns:
        Gurobi expression (LinExpr, QuadExpr, or constant).
    """
    try:
        # Create a temporary variable to hold the result
        temp_expr = sympy_expr.simplify()
        temp_expr = temp_expr.apart()

        # Only if both operations succeed, update sympy_expr
        sympy_expr = temp_expr
    except Exception as e:
        # Handle the exception and continue with the old value
        # print(f"An error occurred: {e}")
            a=5
    if isinstance(sympy_expr, sp.Symbol):
        return symbol_map[sympy_expr]
    
    elif isinstance(sympy_expr, sp.Add):
        return sum(sympy_to_gurobi(arg, symbol_map, model, aux_var_count) for arg in sympy_expr.args)
    
    elif isinstance(sympy_expr, sp.Mul):
        result = 1
        for arg in sympy_expr.args:
            result *= sympy_to_gurobi(arg, symbol_map, model, aux_var_count)
        return result
    
    elif isinstance(sympy_expr, sp.Pow):
        base, exp = sympy_expr.args
        
        # Always create an auxiliary variable for the base
        base_expr = sympy_to_gurobi(base, symbol_map, model, aux_var_count)
        aux_var_name = f"pow_base_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        base_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        model.addConstr(base_aux_var == base_expr)
        

        if isinstance(exp, sp.Number):
            # Handle non-quadratic powers using general constraints
            exp_value = float(exp)
            aux_var_name = f"pow_aux_{aux_var_count[0]}"
            aux_var_count[0] += 1
            pow_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
            model.addGenConstrPow(base_aux_var, pow_aux_var, exp_value)
            return pow_aux_var
        else:
            # Handle symbolic powers using general constraints
            aux_var_name = f"pow_aux_{aux_var_count[0]}"
            aux_var_count[0] += 1
            pow_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)

            # Convert the base to a Gurobi expression
            base_expr = sympy_to_gurobi(base, symbol_map, model, aux_var_count)

            # Create an auxiliary variable for the base
            base_aux_var_name = f"base_aux_{aux_var_count[0]}"
            aux_var_count[0] += 1
            base_aux_var = model.addVar(name=base_aux_var_name, vtype=GRB.CONTINUOUS)
            model.addConstr(base_aux_var == base_expr)

            # Create an auxiliary variable for the logarithm of the base
            log_base_aux_var_name = f"log_base_aux_{aux_var_count[0]}"
            aux_var_count[0] += 1
            log_base_aux_var = model.addVar(name=log_base_aux_var_name, vtype=GRB.CONTINUOUS)
            model.addGenConstrLog(base_aux_var, log_base_aux_var)

            # Convert the exponent to a Gurobi expression
            exp_expr = sympy_to_gurobi(exp, symbol_map, model, aux_var_count)

            # Create an auxiliary variable for the product of the exponent and the logarithm of the base
            prod_aux_var_name = f"prod_aux_{aux_var_count[0]}"
            aux_var_count[0] += 1
            prod_aux_var = model.addVar(name=prod_aux_var_name, vtype=GRB.CONTINUOUS)
            model.addConstr(prod_aux_var == exp_expr * log_base_aux_var)

            # Create the exponential constraint
            model.addGenConstrExp(prod_aux_var, pow_aux_var)

            return pow_aux_var
        
    
    elif isinstance(sympy_expr, sp.exp):
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        aux_var_name = f"exp_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", lb=0, vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        exp_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        model.addGenConstrExp(arg_aux_var, exp_aux_var)
        return exp_aux_var
    
    elif isinstance(sympy_expr, sp.log):
        arg_expr = sympy_to_gurobi(sympy_expr.args[0], symbol_map, model, aux_var_count)
        aux_var_name = f"log_aux_{aux_var_count[0]}"
        aux_var_count[0] += 1
        arg_aux_var = model.addVar(name=f"aux_{aux_var_name}_arg", vtype=GRB.CONTINUOUS)
        model.addConstr(arg_aux_var == arg_expr)
        log_aux_var = model.addVar(name=aux_var_name, vtype=GRB.CONTINUOUS)
        model.addGenConstrLog(arg_aux_var, log_aux_var)
        return log_aux_var


    elif isinstance(sympy_expr, sp.Number):
        return float(sympy_expr)
    
    else:
        raise ValueError(f"Unsupported SymPy expression: {sympy_expr}")

In [12]:

#Generate one regulator
x = sp.symbols('x')

class Regulator:
    _id_counter = 1

    def __init__(self, name, permit_price, emission_cap):
        self.id = Regulator._id_counter
        Regulator._id_counter += 1
        self.name = name
        self.permit_price = permit_price
        self.emission_cap = emission_cap

    def __repr__(self):
        return f"Regulator(id={self.id}, name='{self.name}', permit_price={self.permit_price}, emission_cap={self.emission_cap})"

# Global registries for sectors and countries
sector_registry = {}
country_registry = {}
firm_registry = {}
class Sector:
    _id_counter = 1

    def __init__(self, name, price_demand_function, free_emission_multiplier):
        self.id = Sector._id_counter
        Sector._id_counter += 1
        self.name = name
        self.price_demand_function = price_demand_function
        self.free_emission_multiplier = free_emission_multiplier
        self.firms = []  # List to store firms in this sector

        # Register this sector in the global registry
        sector_registry[self.id] = self

    def add_firm(self, firm):
        self.firms.append(firm)

    def __repr__(self):
        return f"Sector(id={self.id}, name='{self.name}', firms={len(self.firms)})"

class Country:
    _id_counter = 1

    def __init__(self, name, size):
        self.id = Country._id_counter
        Country._id_counter += 1
        self.name = name
        self.size = size
        self.firms = []  # List to store firms in this country

        # Register this country in the global registry
        country_registry[self.id] = self

    def add_firm(self, firm):
        self.firms.append(firm)

    def __repr__(self):
        return f"Country(id={self.id}, name='{self.name}', firms={len(self.firms)})"

class Firm:
    _id_counter = 1

    def __init__(self, name, sector, country, production_cost_function, abatement_cost_function, actual_output, emission, profit):
        self.id = Firm._id_counter
        Firm._id_counter += 1
        self.name = name

        # Determine sector and country from either object or ID
        if isinstance(sector, Sector):
            self.sector = sector
        elif isinstance(sector, int):
            self.sector = sector_registry.get(sector)

        if isinstance(country, Country):
            self.country = country
        elif isinstance(country, int):
            self.country = country_registry.get(country)

        self.production_cost_function = production_cost_function
        self.abatement_cost_function = abatement_cost_function
        self.actual_output = actual_output
        self.emission = emission
        self.profit = profit

        # Register this firm in the global registry
        firm_registry[self.id] = self
        
        # Register this firm with its sector and country
        if self.sector:
            self.sector.add_firm(self)
        if self.country:
            self.country.add_firm(self)

    def __repr__(self):
        return f"Firm(id={self.id}, name='{self.name}', sector_id={self.sector.id if self.sector else None}, country_id={self.country.id if self.country else None}, actual_output={self.actual_output}, emission={self.emission}, profit={self.profit})"
    


In [13]:
# Example Usage
import sympy as sp
x = sp.symbols('x')
# Create Sectors and Countries
Regulator1 = Regulator('EU', permit_price= 2, emission_cap= 500)
sector1 = Sector('cement', price_demand_function= 10 - 0.1*x, free_emission_multiplier= 0.1)
sector2 = Sector('steel', price_demand_function=20 - 0.1*x, free_emission_multiplier= 0.02)
sector3 = Sector('paper', price_demand_function= 100 - 0.02*x**1.5, free_emission_multiplier= 0.1)
country1 = Country('DE', 1)
country2 = Country('FI', 0.5)
country3 = Country('GR', size= 0.1)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 0.1*sp.exp('x')**2, 0, 0, 0)
firm2 = Firm('firm2', 1, 2, x*0, 5*x**2, 0, 0, 0)
firm3 = Firm('firm3', 1, 3, x*0, x**1.5, 0, 0, 0)
firm4 = Firm('firm4', 2, 1, x*0, x**1.1, 0, 0, 0)
firm5 = Firm('firm5', 2, 2, x*0, x**1.2, 0, 0, 0)
firm6 = Firm('firm6', 2, 3, x*0, x**1.35, 0, 0, 0)
firm7 = Firm('firm7', 3, 1, x*0, x**1.4, 0, 0, 0)
firm8 = Firm('firm8', 3, 2, x*0, x**1.3, 0, 0, 0)
firm9 = Firm('firm9', 3, 3, x*0, x+x**1.2, 0, 0, 0)


print(sector1.firms)  # List of firms in sector1
print(country1.firms)  # List of firms in country1
print(firm_registry)  # Dictionary of all firms

for firm in firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name)

[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0)]
[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=7, name='firm7', sector_id=3, country_id=1, actual_output=0, emission=0, profit=0)]
{1: Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), 2: Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), 3: Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0), 4: Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), 5: Firm(id=5, name='firm5', sector_id=2, country_id=2, actual_out

In [14]:
def calculate_output(firms, regulator, verbose=False, writeLP=True):

    m = gb.Model("Firm Optimization")
    # Dictionaries to hold SymPy symbols and Gurobi variables
    out_symbols = {}
    em_symbols = {}
    output_vars = {}
    emission_vars = {}
    profit_functions = {}
    combined_profit_expr = 0  # Combined profit expression
    price_symbols = {}
    price_symbolic_expressions = {}
    price_expressions = {}
    price_vars = {}


    
    # Create all the variables
    for firm in firms:
        out_symbol = sp.Symbol(f'out_{firm.id}')
        em_symbol = sp.Symbol(f'em_{firm.id}')
        out_symbols[firm.id] = out_symbol
        em_symbols[firm.id] = em_symbol
        # Create Gurobi variables for output and emission
        output_var = m.addVar(vtype=gb.GRB.CONTINUOUS, name=f"output_{firm.id}", lb=0)
        emission_var = m.addVar(vtype=gb.GRB.CONTINUOUS, name=f"emission_{firm.id}", lb=0)
        m.addConstr(emission_var <= output_var)
        output_vars[firm.id] = output_var
        emission_vars[firm.id] = emission_var

    for sector in set(firm.sector for firm in firms):
        firms_in_sector = set(sector.firms) & set(firms)
        price_symbol = sp.Symbol(f'price_{sector.id}')
        price_symbols[sector.id] = price_symbol
        price_var = m.addVar(vtype=gb.GRB.CONTINUOUS, name=f"price_{sector.id}", lb=0)
        price_vars[sector.id] = price_var
        price_symbolic_expressions[sector.id] = sector.price_demand_function.subs(x, sum(out_symbols[firm.id] for firm in firms_in_sector))
        mapped_dict = {out_symbols[key]: output_vars[key] for key in output_vars}
        price_expressions[sector.id] = sympy_to_gurobi(price_symbolic_expressions[sector.id], mapped_dict, m)
        m.addConstr(price_var == price_expressions[sector.id])


    #Get the price of the permits
    permit_price = regulator.permit_price
    for firm in firms:
        # Calculate the output of the firm
        sector = firm.sector
        # Sum of all other outputs in the same sector
        
        # Get the price demand function of the sector
        # price_demand_function = sector.price_demand_function
        # Get the abatement cost function of the firm
        abatement_cost_function = firm.abatement_cost_function
        # Get the production cost function of the firm
        production_cost_function = firm.production_cost_function
        # Get the free emission multiplier of the firm
        free_emission_multiplier = sector.free_emission_multiplier


        out_symbol = out_symbols[firm.id]
        em_symbol = em_symbols[firm.id]
        
        # Calculate the output of the firm


        #print("Price to demand Function: {}".format(price_demand_function.subs(x, sum_other_outputs + out_symbol)))
        income = (price_symbols[sector.id] - production_cost_function.subs(x, out_symbol))*out_symbol
        abatement = -abatement_cost_function.subs(x, out_symbol - em_symbol)
        trading = - permit_price * (em_symbol -free_emission_multiplier * out_symbol)
        profit_expr = income + abatement + trading

        #     # Create Gurobi variables for output and emission
        # output_var = m.addVar(vtype=gb.GRB.CONTINUOUS, name=f"output_{firm.id}", lb=0)
        # emission_var = m.addVar(vtype=gb.GRB.CONTINUOUS, name=f"emission_{firm.id}", lb=0)
        # m.addConstr(emission_var <= output_var)
        #symbol_map = {out_symbol: output_var, em_symbol: emission_var}


        mapped_out = {out_symbols[key]: output_vars[key] for key in output_vars}
        mapped_em = {em_symbols[key]: emission_vars[key] for key in emission_vars}
        mapped_price = {price_symbols[key]: price_vars[key] for key in price_vars}
        symbol_map = {**mapped_out, **mapped_em, **mapped_price}

        profit = sympy_to_gurobi(profit_expr, symbol_map, m)
        profit_functions[firm.id] = profit_expr
        combined_profit_expr += profit
    

    m.setObjective(combined_profit_expr, gb.GRB.MAXIMIZE) 
    m.params.OutputFlag = 1 if verbose else 0
    m.optimize()
    if writeLP:
        m.write("03.lp".format(writeLP))
    # Print the results
    results = pd.DataFrame(columns=["Firm", "Output", "Emission", "Profit"])
    rows = []  # List to collect rows
    for firm in firms:
        out_symbol = out_symbols[firm.id]
        em_symbol = em_symbols[firm.id]
        output_var = output_vars[firm.id]
        emission_var = emission_vars[firm.id]
        price_var = price_vars[firm.sector.id]
        firm.actual_output = output_var.x
        firm.emission = emission_var.x
        firm.profit = profit_functions[firm.id].subs({out_symbol: output_var.x, em_symbol: emission_var.x, price_symbols[firm.sector.id]: price_var.x})
        rows.append({"Firm": firm.name, "Output": output_var.x, "Emission": emission_var.x, "Profit": firm.profit})
        print(f"{firm.name}: Output: {output_var.x:.2f}, Emission: {emission_var.x:.2f}, Profit: {firm.profit:.2f}")

    # Convert the list of rows to a DataFrame and concatenate with the original DataFrame
    results = pd.concat([results, pd.DataFrame(rows)], ignore_index=True)    
    return results
    
#print(calculate_output([firm1,firm3], Regulator1))
results = calculate_output([firm1,firm2,firm3,firm4,firm5,firm6,firm7,firm8,firm9], Regulator1, verbose=True)



# Example Usage
# output, emission, profit = calculate_output([firm1,firm2], Regulator1)
# print(f"Output: {output}, Emission: {emission}, Profit: {profit}")

Set parameter Username


Academic license - for non-commercial use only - expires 2025-08-14
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-9900 CPU @ 3.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22 rows, 41 columns and 59 nonzeros
Model fingerprint: 0x19d88e0f
Model has 9 quadratic objective terms
Model has 10 general constraints
Variable types: 41 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+00]
  Objective range  [4e-02, 5e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve added 97 rows and 664 columns
Presolve time: 0.39s
Presolved: 132 rows, 712 columns, 8968 nonzeros
Presolved model has 6 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 663 continuous, 49 integer (13 binary)
Found heuristic solution: objective 1023.47014

C:\Users\Kostas\AppData\Local\Temp\ipykernel_14620\805260941.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame(rows)], ignore_index=True)


In [15]:
# print pandas dataframe for latex output
#convert instances of Firm to a pandas dataframe
df = pd.DataFrame([vars(sector) for sector in sector_registry.values()])
#for firm in firm_registry.values():
print(df.to_latex(index=True))


\begin{tabular}{lrllrl}
\toprule
 & id & name & price_demand_function & free_emission_multiplier & firms \\
\midrule
0 & 1 & cement & 10 - 0.1*x & 0.100000 & [Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=15.344695272504032, emission=14.181633910619171, profit=64.2049257163978), Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=17.483278063222304, emission=17.259892908706885, profit=71.8665933252161), Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=8.178955941671937, emission=6.369844243611823, profit=34.7129628397522)] \\
1 & 2 & steel & 20 - 0.1*x & 0.020000 & [Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=83.55882970768025, emission=0.0, profit=779.523753169582), Firm(id=5, name='firm5', sector_id=2, country_id=2, actual_output=5.972714403801901, emission=0.0, profit=56.4783185849649), Firm(id=6, name='firm6', sector_id=2, country_id=3, actual_output=2.011249853718846, emission=0.0, profit=19.3254066645350)] \